## 5.2 PyTorch的nn模块

PyTorch有一个专门用于**神经网络的完整子模块：torch.nn。**该子模块包含创建各种神经网络体系结构所需的构建块。这些构建块在PyTorch术语中称为module（模块），在其他框架中称为layer（层）。

**PyTorch模块都是从基类nn.Module继承而来的Python类**。模块可以具有一个或多个**参数（Parameter）实例作为属性**，这些**参数就是在训练过程中需要优化的张量**（在之前的线性模型中即w和b）。模块还可以具有一个或多个子模块（nn.Module的子类）属性，并且也可以**追踪其参数**。

注：子模块必须是顶级属性（top-level attributes），而不能包含在list或dict实例中！否则，优化器将无法找到子模块（及其参数）。对于需要子模块列表或字典的情况，PyTorch提供有nn.ModuleList和nn.ModuleDict。

---

In [251]:
# 所有PyTorch提供的nn.Module子类都定义了其调用方法，
    #使你可以实例化nn.Linear并将其像一个函数一样进行调用# 
import torch 
import torch.nn as nn

x = torch.randn(3,5)
    # 生成输入张量 # 参数: 尺寸（输入样本个数，每个输入样本的大小）    
    
linear_model = nn.Linear(5,7,bias=True)  
    # 通过调用nn.Linear构造了一个Model
        #  参数: 输入样本的大小, 输出样本的大小, 是否包含偏差bias  
            # 注意：参数是样本的大小，而不是样本个数。 
        
y = linear_model(x)
    # 使用一组参数调用nn.Module实例及其子类的实例
        # 最终会调用带有相同参数的名为forward的方法
    # 参数：输入样本集
    # 注意：不要这样做 , y = linear_model.forward(x)  
        # 虽然的确可以调用forward，但是很多hook则无法调用

print('x:\n',x),print('linear_model:\n',linear_model),print('y:\n',y)

x:
 tensor([[ 1.1972, -0.3102,  0.9951, -1.4386,  0.8193],
        [ 0.0500,  0.4547,  0.2050, -0.1255, -1.6829],
        [-1.2626, -0.9231, -1.4173,  1.9514,  0.8499]])
linear_model:
 Linear(in_features=5, out_features=7, bias=True)
y:
 tensor([[ 0.7737,  0.5143,  0.0317, -0.8034, -0.3835,  0.5370, -0.7645],
        [-0.4224, -0.6322,  0.3459,  0.4946,  0.9803, -0.3622, -0.3927],
        [ 1.0827, -0.4843,  0.4638,  0.4160,  0.0366,  0.5097,  0.7585]],
       grad_fn=<AddmmBackward>)


(None, None, None)

#### 为什么nn.Model及其子实例能调用forward方法呢？ 
内部源码：这里是Module.call的实现
```
def __call__(self, *input, **kwargs):
    for hook in self._forward_pre_hooks.values():
        hook(self, input)
    result = self.forward(*input, **kwargs)
    for hook in self._forward_hooks.values():
        hook_result = hook(self, input, result)
        # ...
    for hook in self._backward_hooks.values():
        # ...
    return result
```

In [252]:
# 块可以具有一个或多个参数（Parameter）实例作为属性
linear_model.weight,linear_model.bias 

(Parameter containing:
 tensor([[ 0.2270, -0.2665,  0.1763,  0.3294,  0.4097],
         [ 0.0500, -0.2792,  0.3660, -0.0502,  0.2080],
         [-0.2435,  0.1715,  0.0469,  0.0149,  0.0315],
         [-0.3035,  0.2968, -0.3312, -0.0407, -0.2056],
         [-0.0731,  0.0729,  0.4143,  0.3827, -0.4192],
         [ 0.1087, -0.2614, -0.1754, -0.1033,  0.2308],
         [-0.2497, -0.3100, -0.0637,  0.1669, -0.0212]], requires_grad=True),
 Parameter containing:
 tensor([ 0.3821, -0.2390,  0.3253,  0.0915,  0.2084,  0.1626, -0.2409],
        requires_grad=True))

---

#### 成批处理。
1：充分利用执行计算的计算资源.2：为了某些高级模型将使用整个批次的统计信息，而当批次大小较大时那些统计数据将变得更准确。

注意：仅对input进行分批，target需要为1D

In [253]:
# 我们有以下数据集，
    # 但是其生成的张量的尺寸是一维的
y = [0.5,  14.0, 15.0, 28.0, 11.0,  8.0,  3.0, -4.0,  6.0, 13.0, 21.0]
x = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4]
y = torch.tensor(y) 
x = torch.tensor(x) 
x.shape,y.shape

(torch.Size([11]), torch.Size([11]))

In [254]:
# 我们将张量reshape为二维的。  
    # 2D：BxNin 增加了Nin这个维度在dim=1的方向上。  # batch 和 num_of_input 的缩写
x = x.unsqueeze(1) 
# y = y.unsqueeze(dim=1) target is must 1D
x.shape,y.shape

(torch.Size([11, 1]), torch.Size([11]))

---

In [255]:
# nn.Module或其子模块的参数列表
        # 所以：优化器构造函数的参数是[params]，我们可以用该方法来提供。
list(linear_model.parameters())
    # 强调一下：模块本身可以具有一个或多个参数（Parameter）实例作为属性。
        # 所以parameters方法的原理是：
            # 递归调用到模块的init构造函数中定义的子模块中，并返回遇到的所有参数的列表，

[Parameter containing:
 tensor([[ 0.2270, -0.2665,  0.1763,  0.3294,  0.4097],
         [ 0.0500, -0.2792,  0.3660, -0.0502,  0.2080],
         [-0.2435,  0.1715,  0.0469,  0.0149,  0.0315],
         [-0.3035,  0.2968, -0.3312, -0.0407, -0.2056],
         [-0.0731,  0.0729,  0.4143,  0.3827, -0.4192],
         [ 0.1087, -0.2614, -0.1754, -0.1033,  0.2308],
         [-0.2497, -0.3100, -0.0637,  0.1669, -0.0212]], requires_grad=True),
 Parameter containing:
 tensor([ 0.3821, -0.2390,  0.3253,  0.0915,  0.2084,  0.1626, -0.2409],
        requires_grad=True)]

In [256]:
# 构造优化器
optimizer = torch.optim.Adam(
    params=linear_model.parameters(),  # 使用parameters方法获取任何nn.Module或其子模块的参数列表
    lr=1e-2)
    # Adam优化器
    # 参数列表是require_grad = True的张量列表，所以优化器可以追踪这些参数。
         #当调用raining_loss.backward()时，grad将累积在图的叶节点上，
         #当调用optimizer.step()时，它将循环访问每个参数，并按与存储在其grad属性中的值成比例的量对其进行更改。
         #在更新参数之后，下一次的参数列表，将再次传递给优化器。    

In [257]:
# 现成的损失函数
    # 不再使用自己定义的loss 
loss_fn = nn.MSELoss() # MSELoss均方误差 # 还有交叉熵损失 CrossEntropyLoss
loss_fn

MSELoss()

In [258]:
# 定义model
    # 用神经网络代替线性模型作为近似函数
        # 通过nn.Sequential容器串联模块

seq_model = nn.Sequential(
            nn.Linear(1, 13),  # 作为神经网络模型的输入模块(层)
            nn.ReLU(),         # 定义的激活函数torch.nn.ReLU（建议） , 类似的还有torch.nn.Sigmoid  
            nn.Linear(13, 1))  # 作为神经网络模型的输出模块(层)
            
            # 得到的模型的输入是作为nn.Sequential的参数的第一个模块所指定的输入，
                # 然后将中间输出传递给后续模块，并输出最后一个模块返回的输出。
            # 该模型将1个输入特征散开为13个隐藏特征，然后将通过ReLU激活函数，
                # 最后将得到的13个数字线性组合为1个输出特征。
seq_model

Sequential(
  (0): Linear(in_features=1, out_features=13, bias=True)
  (1): ReLU()
  (2): Linear(in_features=13, out_features=1, bias=True)
)

In [259]:
#训练循环几乎没有改变，除了现在你不再明确地将params传递给model，
        # 因为model本身在内部保存有Parameters , 用parameters方法就能得到参数列表
def training_loop(n_epochs, optimizer, model, loss_fn, 
                  x_train, x_val, y_train, y_val,
                  verbose=500):
    for epoch in range(1, n_epochs + 1):
        p_train = model(xn_train)  # xn中的n是归一化normalization的缩写 
        loss_train = loss_fn(p_train, y_train)

        p_val = model(xn_val)
        loss_val = loss_fn(p_val, y_val)

        optimizer.zero_grad()  # 优化器清零grad
        loss_train.backward() # loss_train向后传播
        optimizer.step() # 优化器更新权置参数

        if epoch == 1 or epoch % 1000 == 0:
            print('Epoch %d, Training loss %.4f, Validation loss %.4f' % (
                    epoch, float(loss_train), float(loss_val)))

In [260]:
# 调用model.parameters()可以得到第一线性模块和第二线性模块中的权重和偏差。

[params for param in seq_model.parameters()]  
    # 再次强调:
        # 使用parameters方法获取任何nn.Module或其子模块实例的参数列表
            # 这些参数列表都是优化器所需的参数张量，
                #在调用loss_train.backward()之后，所有参数都将被计算其grad，
                    # 然后优化器会在调用optimizer.step()期间更新参数的值

[tensor([1., 0.]), tensor([1., 0.]), tensor([1., 0.]), tensor([1., 0.])]

In [261]:
# nn.Modules参数的一些注意事项
   # named_parameters方法 
        # 当你检查有几个子模块组成的模型的参数时，方便地通过名称识别参数。
        
for name, param in seq_model.named_parameters():
    print(name,param.shape)
    # 可以看见，这里的模块名称全是数字
        # 因为这里每个模块名称默认就是定义Sequential时的模块顺序。
            # 其实，Sequential还可以接受OrderedDict作为参数，这样就可以给Sequential的每个模块命名

0.weight torch.Size([13, 1])
0.bias torch.Size([13])
2.weight torch.Size([1, 13])
2.bias torch.Size([1])


In [262]:
# 这里每个模块名称默认就是定义Sequential时的模块顺序。
    # 其实，Sequential还可以接受OrderedDict作为参数，这样就可以给Sequential的每个模块命名



In [263]:
from collections import OrderedDict  # ordered 有序的
    # 我发觉OrderedDict这个玩意就像是装饰器一样，包装了一些”华丽花哨“的东西。

from collections import OrderedDict

seq_model = nn.Sequential(OrderedDict([
    ('input_linear', nn.Linear(1, 8)),
    ('hidden_activation', nn.Tanh()),
    ('output_linear', nn.Linear(8, 1))
]))

seq_model


Sequential(
  (input_linear): Linear(in_features=1, out_features=8, bias=True)
  (hidden_activation): Tanh()
  (output_linear): Linear(in_features=8, out_features=1, bias=True)
)

In [264]:
# OrderedDict之后
for name, param in seq_model.named_parameters():
    print(name, param.shape)


input_linear.weight torch.Size([8, 1])
input_linear.bias torch.Size([8])
output_linear.weight torch.Size([1, 8])
output_linear.bias torch.Size([1])


In [265]:
# 通过子模块来访问特定参数，就像它们是属性一样 
seq_model.output_linear.bias


Parameter containing:
tensor([0.0705], requires_grad=True)

In [266]:
# 可以在训练循环代码 中打印如下信息
# print('output_val', seq_model(xn_val))
# print('target_val', y_val)
# print('hidden', seq_model.input_linear.weight.grad)